In [1]:
import pandas as pd
import numpy as np
import ssl
import urllib.request, urllib.parse, urllib.error
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import glob
import os.path
import re
from pandas.errors import EmptyDataError



In [2]:
rochelle = pd.read_excel(r"C:\Users\justi\Downloads\GLANSIS Spp. Tracking.xlsx", sheet_name = "Listed")


In [3]:
#rochelle.columns
namelist= []
for name in rochelle['Scientific Name']:
    namelist.append(name)

print(len(namelist))

302


In [4]:
# Ignore SSL certificate errors --from Beautiful Soup SI 206 textbook
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [5]:
#accessing search page of the herbarium
atlas_url = "https://bellatlas.umn.edu/collections/harvestparams.php"
atlas_html = urllib.request.urlopen(atlas_url, context=ctx).read()
#herbarium_html


In [6]:
#setting up driver for Selenium
driver = webdriver.Firefox() #???Should I write a script for multiple Browsers?
rounds = 0
emptyresults = 0
masterdataframe = pd.DataFrame()
driver.maximize_window()
scraped_dataframe = pd.DataFrame()#(index=[0])
scraped_dataframe.reset_index(drop=True)
for speciesname in namelist[:]:
    #print(speciesname)
    driver.get(atlas_url) # Goes to atlas
    namebox = driver.find_element(by = By.XPATH, value = '//*[@id="taxa"]')
    searchbutton = driver.find_element(by = By.XPATH, value='/html/body/table/tbody/tr/td/div[4]/form/div[2]/div[1]/button') #selects the advanced search button based off the pulled tag from right clicking the inspected element
    time.sleep(.5)
    namebox.clear()
    namebox.send_keys(speciesname) #'Cabomba caroliniana') #Sends in a name from Rochelle's list
    searchbutton.click()

    resultcount = driver.find_element(by=By.XPATH, value='/html/body/table/tbody/tr/td/div[4]/div/div[2]/div/div[4]/div[3]').text
    #print(resultcount) #Page 1, records 1-64 of 64
    resultcountnumberlist= list(map(int, list(filter(lambda x: x!= '', re.findall(r'\d*', re.search(r'\d* of \d*', resultcount).group()))))) #first number in list is the total results displayed on the page, the second number is the total results
    #print(resultcountnumberlist)
#if there are not results
    if resultcountnumberlist[1] == 0:
        emptyresults+=1
        continue
#continue to next search
    
#if there are results
    windowbefore = driver.window_handles[0]
    #resultcount = driver.find_element(by = By.XPATH, value ='/html/body/table/tbody/tr/td/div[4]/div/div[2]/div/div[4]/div[3]').text
    downloadbutton = driver.find_element(by=By.XPATH, value='/html/body/table/tbody/tr/td/div[4]/div/div[2]/div/div[1]/form/button')
    downloadbutton.click()
    windowafter = driver.window_handles[1]
    driver.switch_to.window(windowafter)
    time.sleep(1)
    driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/form/fieldset/table/tbody/tr[4]/td[2]/div/input[2]').click() #utf8 encoding
    driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/form/fieldset/table/tbody/tr[5]/td[2]/div/input').click() #uncompresses download

    downloadbutton=driver.find_element(by=By.NAME, value= "submitaction") #OVERWRITES THE PREVIOUS DOWNLOAD BUTTON
    downloadbutton.click()

    folder_path = r'C:\Users\justi\Downloads'
    file_type = r'\*CSV'
    files = glob.glob(folder_path + file_type)
    max_file = max(files, key=os.path.getctime)

    try:
        downloadeddata = pd.read_csv(max_file, encoding='utf-8')
    except EmptyDataError:
        print('Empty Data Error Occured on file ' + str(max_file))
        downloadeddatafile = open(max_file, 'r')
        downloadeddata = pd.read_csv(downloadeddatafile, encoding='utf-8')
        downloadeddatafile.close()
        continue
    except:
        print("Skipping to next interation")
        continue
        
    
    if masterdataframe.empty:
        masterdataframe = pd.DataFrame(data=downloadeddata)
        #print('Empty')
    else:
        masterdataframe = masterdataframe.append(pd.DataFrame(data = downloadeddata))
    driver.switch_to.window(windowbefore)
    rounds +=1
    os.remove(max_file)
    

Empty Data Error Occured on file C:\Users\justi\Downloads\1681413556-occur.csv


EmptyDataError: No columns to parse from file

In [ ]:
print(rounds)
masterdataframe

124


,id,institutionCode,collectionCode,ownerInstitutionCode,basisOfRecord,occurrenceID,catalogNumber,otherCatalogNumbers,kingdom,phylum,...,verbatimDepth,verbatimElevation,disposition,language,recordEnteredBy,modified,sourcePrimaryKey-dbpk,collId,recordId,references
0,6318,MIN,Plants,NaN,PreservedSpecimen,8aa59e90-b0b1-4085-ac65-7d396f385597,911585,1134888,Plantae,Spermatophyta,...,NaN,NaN,NaN,NaN,NaN,2018-02-27 10:36:53,NaN,1,urn:uuid:8aa59e90-b0b1-4085-ac65-7d396f385597,https://bellatlas.umn.edu:443/collections/indi...
1,7662,MIN,Plants,NaN,PreservedSpecimen,c459d154-b6d7-4eb8-a1db-91d79c8ee8b6,906278,1134887,Plantae,Spermatophyta,...,NaN,NaN,NaN,NaN,NaN,2018-02-27 10:36:53,NaN,1,urn:uuid:c459d154-b6d7-4eb8-a1db-91d79c8ee8b6,https://bellatlas.umn.edu:443/collections/indi...
2,10966,MIN,Plants,NaN,PreservedSpecimen,a5d03247-7ffa-4f8e-af0b-be7e3210c806,906219,1134883,Plantae,Spermatophyta,...,NaN,NaN,NaN,NaN,NaN,2018-02-27 10:36:53,NaN,1,urn:uuid:a5d03247-7ffa-4f8e-af0b-be7e3210c806,https://bellatlas.umn.edu:443/collections/indi...
3,16127,MIN,Plants,NaN,PreservedSpecimen,45527e00-3766-4b45-afff-93d99f5e90e1,477374,NaN,Plantae,Spermatophyta,...,NaN,NaN,NaN,NaN,NaN,2018-02-27 10:36:53,NaN,1,urn:uuid:45527e00-3766-4b45-afff-93d99f5e90e1,https://bellatlas.umn.edu:443/collections/indi...
4,18640,MIN,Plants,NaN,PreservedSpecimen,903158b0-1dff-44a0-9785-01017e8b0744,427253,1134886,Plantae,Spermatophyta,...,NaN,NaN,NaN,NaN,NaN,2018-02-27 10:36:53,NaN,1,urn:uuid:903158b0-1dff-44a0-9785-01017e8b0744,https://bellatlas.umn.edu:443/collections/indi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,231401,JFBM,Fishes,NaN,PreservedSpecimen,910c6e92-6413-11e3-81f0-12313d0829ba,37701,NaN,Animalia,Chordata,...,NaN,NaN,NaN,NaN,NaN,2018-03-13 12:02:38,NaN,4,urn:uuid:910c6e92-6413-11e3-81f0-12313d0829ba,https://bellatlas.umn.edu:443/collections/indi...
5,232360,JFBM,Fishes,NaN,PreservedSpecimen,9139c84c-6413-11e3-81f0-12313d0829ba,41280,NaN,Animalia,Chordata,...,NaN,NaN,NaN,NaN,NaN,2018-03-13 12:02:38,NaN,4,urn:uuid:9139c84c-6413-11e3-81f0-12313d0829ba,https://bellatlas.umn.edu:443/collections/indi...
6,234300,JFBM,Fishes,NaN,PreservedSpecimen,918118fa-6413-11e3-81f0-12313d0829ba,42486,NaN,Animalia,Chordata,...,NaN,NaN,NaN,NaN,NaN,2018-03-13 12:02:38,NaN,4,urn:uuid:918118fa-6413-11e3-81f0-12313d0829ba,https://bellatlas.umn.edu:443/collections/indi...
7,236937,JFBM,Fishes,NaN,PreservedSpecimen,91ca6a6e-6413-11e3-81f0-12313d0829ba,38631,NaN,Animalia,Chordata,...,NaN,NaN,NaN,NaN,NaN,2018-03-13 12:02:38,NaN,4,urn:uuid:91ca6a6e-6413-11e3-81f0-12313d0829ba,https://bellatlas.umn.edu:443/collections/indi...


In [ ]:
file= masterdataframe.to_csv(r'C:\Users\justi\Downloads\minnatlas-out.csv', encoding='utf-8')

In [ ]:
clienttemplate = pd.read_excel(r"C:\Users\justi\Downloads\GLANSISSpecimenBulkUploadTemplateTEST.xlsx").drop('Unnamed: 0', axis=1)
clienttemplate['Genus*'] = masterdataframe['genus']
clienttemplate['Species*'] = masterdataframe['specificEpithet']
clienttemplate['Latitude*'] = masterdataframe['decimalLatitude']
clienttemplate['Longitude*'] = masterdataframe['decimalLongitude']
clienttemplate['Source*'] = np.where(clienttemplate['Latitude*'].isna()|clienttemplate['Longitude*'].isna(), '', 'reported')
clienttemplate['Accuracy*'] = np.where(clienttemplate['Source*'] == 'reported', 'accurate', '')
#clienttemplate['Locality*']
clienttemplate['Year*'] = masterdataframe['year']
clienttemplate['Month'] = masterdataframe['month']
clienttemplate['Day'] = masterdataframe['day']
clienttemplate['Collectors'] = masterdataframe['recordedBy']
clienttemplate['Reference ID*'] = masterdataframe['id']
#clienttemplate['Contact'] = CONTACT FOR THE DATABASE
#clienttemplate['Reference 2']= LINK FOR THE DATABASE HOME PAGE
#clienttemplate['Reference 3']= LINK FOR THE DATABASE SPECIMEN PAGE
#clienttemplate['Comments']= habitat, identificantion remarks possibly, etc.
#clienttemplate['Record Type*']